In [4]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import requests
from io import StringIO
import re

# URL of the data
url = 'https://data.giss.nasa.gov/gistemp/station_data_v4_globe/v4.temperature.inv.txt'

# Fetch the data from the URL
response = requests.get(url)
data_text = response.text

# Step 1: Clean up the data format
# Replace "Station Name" with "Station" for better column name format
data_text = data_text.replace("Station Name", "Station")

# Replace all spaces and tabs with semicolons
data_text = re.sub(r"[ \t]+", ";", data_text)

# Recursively remove duplicate semicolons
while ';;' in data_text:
    data_text = data_text.replace(";;", ";")

# Remove trailing semicolons in each line
data_lines = data_text.split('\n')
data_lines = [line.rstrip(';') for line in data_lines]

# Join the cleaned lines back into a single string
cleaned_data_text = "\n".join(data_lines)

# Check the header and correct if necessary
if not cleaned_data_text.startswith("ID;Lat;Lon;Elev-m;Station;BI"):
    cleaned_data_text = cleaned_data_text.replace("D;Lat;Lon;Elev-m;Station;BI", "ID;Lat;Lon;Elev-m;Station;BI", 1)

# Step 2: Read the cleaned data into a Pandas DataFrame
data_io = StringIO(cleaned_data_text)

# Initialize an empty list to collect rows that don't match the expected number of fields
valid_rows = []

# Manually parse the rows to handle inconsistencies
for line in data_io:
    # Split each line by the semicolon delimiter
    fields = line.split(';')
    # Check if the number of fields matches the expected number of columns
    if len(fields) == 6:
        valid_rows.append(fields)

# Convert the list of valid rows into a DataFrame
data = pd.DataFrame(valid_rows, columns=["ID", "Lat", "Lon", "Elev-m", "Station", "BI"])

# Convert the numeric columns from strings to appropriate numeric types
data['Lat'] = pd.to_numeric(data['Lat'], errors='coerce')
data['Lon'] = pd.to_numeric(data['Lon'], errors='coerce')
data['BI'] = pd.to_numeric(data['BI'], errors='coerce')

# Drop rows with NaN values in 'Lat', 'Lon', or 'BI'
data = data.dropna(subset=['Lat', 'Lon', 'BI'])

# Function to get the appropriate Folium color based on the BI value
def get_final_folium_color(bi):
    if bi <= 5:
        return 'blue'        # Low
    elif bi <= 15:
        return 'lightred'    # Substitute for Yellow (since yellow is not directly available)
    elif bi <= 25:
        return 'orange'      # High
    else:
        return 'red'         # Very High

# Allow user to specify multiple country codes
country_codes = ['NL', 'UK', "SE", "EI", "SW", "GM"]  # Example: Add the desired country codes here

# Filter the data for the specified country codes
filtered_data = data[data['ID'].str[:2].isin(country_codes)]

# Initialize a new map centered around the average latitude and longitude of the filtered data
map_center = [filtered_data['Lat'].mean(), filtered_data['Lon'].mean()]
filtered_map = folium.Map(location=map_center, zoom_start=3)

# Create feature groups for each BI level
low_group = folium.FeatureGroup(name='Low (0-5)', show=True)
moderate_group = folium.FeatureGroup(name='Moderate (5-15)', show=True)
high_group = folium.FeatureGroup(name='High (15-25)', show=True)
very_high_group = folium.FeatureGroup(name='Very High (25+)', show=True)

# Add markers to the respective feature groups based on the BI value
for index, row in filtered_data.iterrows():
    color = get_final_folium_color(row['BI'])
    marker = folium.Marker(
        location=[row['Lat'], row['Lon']],
        popup=f"Station: {row['Station']}, BI: {row['BI']}",
        icon=folium.Icon(color=color)
    )
    if row['BI'] <= 5:
        marker.add_to(low_group)
    elif row['BI'] <= 15:
        marker.add_to(moderate_group)
    elif row['BI'] <= 25:
        marker.add_to(high_group)
    else:
        marker.add_to(very_high_group)

# Add all groups to the map
low_group.add_to(filtered_map)
moderate_group.add_to(filtered_map)
high_group.add_to(filtered_map)
very_high_group.add_to(filtered_map)

# Add layer control to toggle different BI levels
folium.LayerControl().add_to(filtered_map)

# Save the map to an HTML file
filtered_map_file_path = "interactive_map_with_country_filters.html"
filtered_map.save(filtered_map_file_path)

# Output the path to the saved map file
filtered_map_file_path


'interactive_map_with_country_filters.html'